## 환경 설정


In [ ]:
!pip install --upgrade openai --quiet

In [ ]:
!pip install PyPDF2
!pip install pytesseract
!pip install pdf2image
!pip install langchain openai pypdf faiss-cpu
!pip install -U langchain-community

In [ ]:
import json
from openai import OpenAI
import os
from google.colab import userdata

MODEL = "gpt-4o-mini"
#gpt api 에서 api key를 발급
client = OpenAI(api_key='')

테스트

In [ ]:
completion = client.chat.completions.create(
  model=MODEL,
  messages=[
    {"role": "system", "content": "You are a helpful assistant. Help me with my math homework!"},
    {"role": "user", "content": "Hello! Could you solve 4+5?"}
  ]
)

print("Assistant: " + completion.choices[0].message.content)

Assistant: Of course! The sum of 4 and 5 is 9.


## langchain으로 pdf parsing

In [ ]:
prompt = """
### 지시사항:
1. PDF 문서를 읽고 주요 내용을 파악합니다.
2. 문서에서 기술과 산업을 분석하는데 중요하다고 생각되는 단어와 용어를 뽑아냅니다.
3. 뽑아낸 단어와 용어와 용어의 설명을 묶어 목록으로 제공합니다.
4. 불필요한 추가적인 설명과 해석을 적지 마십시오.
5. 반드시 한국어로 답변하십시오.

### 주의 사항:
- 일반적인 단어(예를들어 '가정','정의','목차')는 무시하십시오. 당신의 목적은 '기술과 산업을 분석'하는 것에 중점을 두어야합니다.
일반적인 용어는 피하고, 기술과 산업을 분석하는데 필요한 기술과 산업에서 사용하는 단어에 집중하십시오.
- 특정 기관의 이름, 개인 및 기업의 명칭은 무시하십시오.
- 기술명, 산업명, 산업에서 쓰이는 기술의 명칭만을 추출하십시오.
- 재무회계, 영업이익 등의 회계단어는 추출하지 마시오.
- 금융자산, 유동성위험 등과 같은 회계용어는 추출하지 마시오.

### 예시:
  - 결과 :
   소프트 로보틱스 ： 소프트 로보틱스는 유연하고 부드러운 재료를 사용해 만들어진 로봇 기술이다.
   발사체 상단 엔진： 지상에서 발사가 될 때는 비추력보다는 지구 중력을 벗어날 수 있는 강한 추진력을 보장하는 기술

이제, 제공되는 PDF 문서를 분석하고 위의 단계를 따라 중요하다고 생각되는 단어와 용어를 뽑아내 주세요.
"""
def pdf_extraction(texts):
  completion = client.chat.completions.create(
    model=MODEL,
    messages=[
      {"role": "system", "content": prompt},
      {"role": "user", "content": texts}
    ]
  )
  return completion.choices[0].message.content

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
import os
import PyPDF2
from tqdm import tqdm
## 여기에 파일 주소 넣기
pdf_file_obj = open('/content/스마트모빌리티 서비스의 현황 및 발전방안 연구.pdf', 'rb')
pdf_reader = PyPDF2.PdfReader(pdf_file_obj)
num_pages = len(pdf_reader.pages)
detected_text = ''
extract_word = []
for page_num in tqdm(range(num_pages)):
    page_obj = pdf_reader.pages[page_num]
    extract_word.append(pdf_extraction(page_obj.extract_text()))
pdf_file_obj.close()




100%|██████████| 184/184 [14:27<00:00,  4.71s/it]


In [ ]:
import pandas as pd
df = pd.DataFrame(extract_word)
df = pd.DataFrame(df[0])

In [ ]:
df

,0
0,- 스마트 모빌리티 서비스: 교통 및 이동성의 효율성을 증대시키기 위해 기술을 활용...
1,- 스마트모빌리티 서비스: 기술과 데이터 기반으로 실시간으로 교통 서비스를 제공하는...
2,- 스마트모빌리티: 개인형 교통수단 및 새로운 교통서비스를 포함하는 모빌리티의 확장...
3,- 스마트모빌리티 서비스: 개인형교통수단에서 새로운 교통서비스 전반으로 확대되고 있...
4,"- 공유 킥보드: 개인형 교통수단으로 운영되는 전동 킥보드의 공유 서비스로, 높은 ..."
...,...
179,- 공유경제: 자원이나 서비스를 여러 사람이 함께 사용하거나 공유하여 이익을 추구하...
180,- 스마트 모빌리티 서비스: 최신 기술을 적용하여 이동 수단 및 교통 서비스의 효율...
181,- MaaS (Mobility-as-a-Service)： 이동 수단 서비스를 통합하...
182,- 스마트 모빌리티 서비스 : 다양한 교통수단을 연계하여 사용자에게 편리한 이동 서...


In [ ]:
splitted_term = []
splitted_exp = []

# 데이터프레임을 사용한 작업 (df가 사전에 정의되어 있어야 함)
for idx, row in tqdm(df.iterrows()):
    temp = row.str.split('\n').str[0]
    for j in temp:
        try:
            splitted_term.append(j.split(':')[0])
        except:
            splitted_term.append('')
            continue
        try:
            splitted_exp.append(j.split(':')[1])
        except:
            splitted_exp.append('')
            continue

184it [00:00, 2073.08it/s]


In [ ]:
bsplitted_term = []
splitted_exp = []
for idx,row in tqdm(df.iterrows()):
  temp = row.str.split('\n').str[0]
  for j in temp:
    try:
      splitted_term.append(j.split(':')[0])
    except:
      splitted_term.append('')
      continue
    try:
      splitted_exp.append(j.split(':')[1])
    except:
      splitted_exp.append('')
      continue

184it [00:00, 2336.17it/s]


In [ ]:
# Excel 파일로 저장
file_path = '/content/스마트모빌리티 서비스의 현황 및 발전방안 연구.pdf.xlsx'
df_total.to_excel(file_path, index=False)

# Google Colab에서 다운로드 링크 제공
from google.colab import files
files.download(file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_total

,term
0,- 스마트 모빌리티 서비스
1,- 스마트모빌리티 서비스
2,- 스마트모빌리티
3,- 스마트모빌리티 서비스
4,- 공유 킥보드
...,...
363,- 공유경제
364,- 스마트 모빌리티 서비스
365,- MaaS (Mobility-as-a-Service)： 이동 수단 서비스를 통합하...
366,- 스마트 모빌리티 서비스


In [ ]:
# 파일 경로 확인
file_path = os.path.abspath(('/주차수행 로봇의 움직임을 제어하기 위해 한쌍의 휠을.pdf.xlsx'))
print(f"파일이 저장된 경로: {file_path}")

파일이 저장된 경로: /주차수행 로봇의 움직임을 제어하기 위해 한쌍의 휠을.pdf.xlsx


In [ ]:
/content/스마트팩토리분야적용가능한IoT시계열데이터의온디바이스고속이벤트복합분석및동기화기술개발.pdf.xlsx

KeyboardInterrupt: 